In [1]:
%pwd

'c:\\Users\\Fahmi Maulana\\Documents\\Projects\\weather-anomaly-detection\\exploratory-data-analysis\\notebooks'

In [2]:
%cd ../..

c:\Users\Fahmi Maulana\Documents\Projects\weather-anomaly-detection


c:\Users\Fahmi Maulana\.pyenv-win-venv\envs\weather-anomaly-detection\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Import Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import joblib

from tseries.utils.general import LogTime
from tseries.feature_engineering.autoregressive_features import (
    add_lags,
    add_rolling_features,
    add_seasonal_rolling_features,
    add_ewma,
)
from tseries.feature_engineering.temporal_features import (
    add_temporal_features,
    bulk_add_fourier_features
)

from tseries.transforms.target_transformations import AutoStationaryTransformer

c:\Users\Fahmi Maulana\Documents\Projects\weather-anomaly-detection\tseries\utils\data_utils.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Load Dataset

In [4]:
# Dictionary mapping weather codes to descriptions
weather_code_mapping = {
    0: "Clear Sky",
    1: "Mainly Clear",
    2: "Partly Cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing Rime Fog",
    51: "Drizzle: Light",
    53: "Drizzle: Moderate",
    55: "Drizzle: Dense",
    56: "Freezing Drizzle: Light",
    57: "Freezing Drizzle: Dense",
    61: "Rain: Slight",
    63: "Rain: Moderate",
    65: "Rain: Heavy",
    66: "Freezing Rain: Light",
    67: "Freezing Rain: Heavy",
    71: "Snow Fall: Slight",
    73: "Snow Fall: Moderate",
    75: "Snow Fall: Heavy",
    77: "Snow Grains",
    80: "Rain Showers: Slight",
    81: "Rain Showers: Moderate",
    82: "Rain Showers: Violent",
    85: "Snow Showers: Slight",
    86: "Snow Showers: Heavy",
    95: "Thunderstorm: Slight or Moderate",
    96: "Thunderstorm with Hail",
    99: "Thunderstorm with Hail"
}

In [5]:
pd.set_option("display.max_columns", None)
raw_df = pd.read_csv("./data/01_raw/jakarta_daily_weather_data_2010-2023.csv", 
                     date_parser=lambda x: pd.to_datetime(x).tz_convert('Asia/Bangkok'),
                     parse_dates=["datetime"],
                    )

daily_processed_df = raw_df.drop(["rain_sum", "snowfall_sum", "sunrise", "sunset"], axis=1).copy()
daily_processed_df["weather_description"] = daily_processed_df["weather_code"].map(weather_code_mapping)
daily_processed_df.rename(columns={"date":"datetime"}, inplace=True)
daily_processed_df["date"] = daily_processed_df["datetime"].dt.date
daily_processed_df["year"] = daily_processed_df["datetime"].dt.year
daily_processed_df["month"] = daily_processed_df.set_index("datetime").index.strftime('%B')
daily_processed_df = daily_processed_df.set_index("datetime")
daily_processed_df.index.freq = 'D'
daily_processed_df


C:\Users\Fahmi Maulana\AppData\Local\Temp\ipykernel_7140\2476853616.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  raw_df = pd.read_csv("./data/01_raw/jakarta_daily_weather_data_2010-2023.csv",


,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,daylight_duration,sunshine_duration,precipitation_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,weather_description,date,year,month
datetime,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00+07:00,55.0,29.346000,24.346000,26.493912,33.960170,28.426277,30.544830,44896.137,25858.787,4.000000,14.0,15.978486,38.160000,245.528370,16.24,3.553079,Drizzle: Dense,2010-01-01,2010,January
2010-01-02 00:00:00+07:00,63.0,30.696000,23.896000,26.858500,37.097730,28.470013,31.728783,44890.758,32148.900,6.500000,4.0,14.707222,31.319998,237.893750,19.80,4.135429,Rain: Moderate,2010-01-02,2010,January
2010-01-03 00:00:00+07:00,63.0,30.346000,24.246000,26.652250,36.627003,28.546375,31.245909,44884.926,38630.723,7.600000,11.0,12.646200,26.640000,243.816120,17.32,3.662728,Rain: Moderate,2010-01-03,2010,January
2010-01-04 00:00:00+07:00,51.0,30.196000,25.296000,27.308502,35.273200,29.467575,31.453005,44878.650,41568.150,0.900000,5.0,19.319628,34.200000,239.246580,20.43,4.381917,Drizzle: Light,2010-01-04,2010,January
2010-01-05 00:00:00+07:00,63.0,29.796000,24.746000,26.373077,35.096622,28.481304,30.532106,44871.930,24485.676,14.300001,7.0,15.893093,32.039997,246.906390,15.86,3.420594,Rain: Moderate,2010-01-05,2010,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27 00:00:00+07:00,61.0,30.765501,24.715500,27.171751,37.685635,29.997322,32.890606,44917.934,39061.520,5.800000,12.0,13.450708,33.839996,341.221830,20.29,4.149257,Rain: Slight,2023-12-27,2023,December
2023-12-28 00:00:00+07:00,63.0,32.015500,24.315500,27.400917,38.291550,29.685040,33.214333,44915.016,42071.566,9.500000,9.0,14.830076,34.920000,342.135070,23.21,4.722630,Rain: Moderate,2023-12-28,2023,December
2023-12-29 00:00:00+07:00,63.0,30.915500,24.965500,27.521751,38.060226,30.560104,33.526210,44911.582,35891.555,8.100000,11.0,14.494192,36.000000,48.758728,19.28,3.979145,Rain: Moderate,2023-12-29,2023,December


# Feature Engineering
## Feature Lags

In [6]:
lags = (
    (np.arange(14) + 1).tolist()
)

lags

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [7]:
with LogTime():
    daily_processed_df, added_features = add_lags(
        daily_processed_df, lags=lags, column="precipitation_sum", ts_id=None, use_32_bit=True
    )
    
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: precipitation_sum_lag_1,precipitation_sum_lag_2,precipitation_sum_lag_3,precipitation_sum_lag_4,precipitation_sum_lag_5,precipitation_sum_lag_6,precipitation_sum_lag_7,precipitation_sum_lag_8,precipitation_sum_lag_9,precipitation_sum_lag_10,precipitation_sum_lag_11,precipitation_sum_lag_12,precipitation_sum_lag_13,precipitation_sum_lag_14


c:\Users\Fahmi Maulana\Documents\Projects\weather-anomaly-detection\tseries\feature_engineering\autoregressive_features.py:66: UserWarning: Assuming just one unique time series in dataset. If there are multiple, provide `ts_id` argument
  warnings.warn(


## Rolling Features

In [8]:
with LogTime():
    daily_processed_df, added_features = add_rolling_features(
        daily_processed_df,
        rolls=[7, 14],
        column="precipitation_sum",
        agg_funcs=["mean", "std"],
        ts_id=None,
        use_32_bit=True,
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: precipitation_sum_rolling_7_mean,precipitation_sum_rolling_7_std,precipitation_sum_rolling_14_mean,precipitation_sum_rolling_14_std


c:\Users\Fahmi Maulana\Documents\Projects\weather-anomaly-detection\tseries\feature_engineering\autoregressive_features.py:149: UserWarning: Assuming just one unique time series in dataset. If there are multiple, provide `ts_id` argument
  warnings.warn(


## Seasonal Rolling Features

In [9]:
# daily_processed_df, added_features = add_seasonal_rolling_features(
#     daily_processed_df,
#     rolls=[3],
#     seasonal_periods=[365],
#     column="precipitation_sum",
#     agg_funcs=["mean", "std"],
#     ts_id=None,
#     use_32_bit=True,
# )

# print(f"Features Created: {','.join(added_features)}")

## Exponentially weighted moving averages (EWMA) Features

In [10]:
with LogTime():
    daily_processed_df, added_features = add_ewma(
        daily_processed_df, 
        alphas=[0.3, 0.5, 0.8], 
        column="precipitation_sum", 
        ts_id=None, 
        use_32_bit=False
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: precipitation_sum_ewma_alpha_0.3,precipitation_sum_ewma_alpha_0.5,precipitation_sum_ewma_alpha_0.8


c:\Users\Fahmi Maulana\Documents\Projects\weather-anomaly-detection\tseries\feature_engineering\autoregressive_features.py:359: UserWarning: Assuming just one unique time series in dataset. If there are multiple, provide `ts_id` argument
  warnings.warn(


## Temporal Embedding Features

In [12]:
with LogTime():
    daily_processed_df, added_features = add_temporal_features(
        daily_processed_df.reset_index(),
        field_name="datetime",
        frequency="D",
        add_elapsed=True,
        drop=False,
        use_32_bit=True,
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: datetime_Month,datetime_Quarter,datetime_Is_quarter_end,datetime_Is_quarter_start,datetime_Is_year_end,datetime_Is_year_start,datetime_Is_month_start,datetime_WeekDay,datetime_Dayofweek,datetime_Dayofyear,datetime_Elapsed


## Fourier Features

In [21]:
daily_processed_df, added_features = bulk_add_fourier_features(
    daily_processed_df,
    ["datetime_Month", "datetime_Quarter", "datetime_WeekDay", "datetime_Dayofyear"],
    max_values=[12, 4, 6, 366],
    n_fourier_terms=5,
    use_32_bit=True,
)

print(f"Features Created: {','.join(added_features)}")

Features Created: datetime_Month_sin_1,datetime_Month_sin_2,datetime_Month_sin_3,datetime_Month_sin_4,datetime_Month_sin_5,datetime_Month_cos_1,datetime_Month_cos_2,datetime_Month_cos_3,datetime_Month_cos_4,datetime_Month_cos_5,datetime_Quarter_sin_1,datetime_Quarter_sin_2,datetime_Quarter_sin_3,datetime_Quarter_sin_4,datetime_Quarter_sin_5,datetime_Quarter_cos_1,datetime_Quarter_cos_2,datetime_Quarter_cos_3,datetime_Quarter_cos_4,datetime_Quarter_cos_5,datetime_WeekDay_sin_1,datetime_WeekDay_sin_2,datetime_WeekDay_sin_3,datetime_WeekDay_sin_4,datetime_WeekDay_sin_5,datetime_WeekDay_cos_1,datetime_WeekDay_cos_2,datetime_WeekDay_cos_3,datetime_WeekDay_cos_4,datetime_WeekDay_cos_5,datetime_Dayofyear_sin_1,datetime_Dayofyear_sin_2,datetime_Dayofyear_sin_3,datetime_Dayofyear_sin_4,datetime_Dayofyear_sin_5,datetime_Dayofyear_cos_1,datetime_Dayofyear_cos_2,datetime_Dayofyear_cos_3,datetime_Dayofyear_cos_4,datetime_Dayofyear_cos_5


# Export Data

In [22]:
daily_processed_df

,datetime,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,daylight_duration,sunshine_duration,precipitation_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,weather_description,date,year,month,precipitation_sum_lag_1,precipitation_sum_lag_2,precipitation_sum_lag_3,precipitation_sum_lag_4,precipitation_sum_lag_5,precipitation_sum_lag_6,precipitation_sum_lag_7,precipitation_sum_lag_8,precipitation_sum_lag_9,precipitation_sum_lag_10,precipitation_sum_lag_11,precipitation_sum_lag_12,precipitation_sum_lag_13,precipitation_sum_lag_14,precipitation_sum_rolling_7_mean,precipitation_sum_rolling_7_std,precipitation_sum_rolling_14_mean,precipitation_sum_rolling_14_std,precipitation_sum_ewma_alpha_0.3,precipitation_sum_ewma_alpha_0.5,precipitation_sum_ewma_alpha_0.8,datetime_Month,datetime_Quarter,datetime_Is_quarter_end,datetime_Is_quarter_start,datetime_Is_year_end,datetime_Is_year_start,datetime_Is_month_start,datetime_WeekDay,datetime_Dayofweek,datetime_Dayofyear,datetime_Elapsed,datetime_Month_sin_1,datetime_Month_sin_2,datetime_Month_sin_3,datetime_Month_sin_4,datetime_Month_sin_5,datetime_Month_cos_1,datetime_Month_cos_2,datetime_Month_cos_3,datetime_Month_cos_4,datetime_Month_cos_5,datetime_Quarter_sin_1,datetime_Quarter_sin_2,datetime_Quarter_sin_3,datetime_Quarter_sin_4,datetime_Quarter_sin_5,datetime_Quarter_cos_1,datetime_Quarter_cos_2,datetime_Quarter_cos_3,datetime_Quarter_cos_4,datetime_Quarter_cos_5,datetime_WeekDay_sin_1,datetime_WeekDay_sin_2,datetime_WeekDay_sin_3,datetime_WeekDay_sin_4,datetime_WeekDay_sin_5,datetime_WeekDay_cos_1,datetime_WeekDay_cos_2,datetime_WeekDay_cos_3,datetime_WeekDay_cos_4,datetime_WeekDay_cos_5,datetime_Dayofyear_sin_1,datetime_Dayofyear_sin_2,datetime_Dayofyear_sin_3,datetime_Dayofyear_sin_4,datetime_Dayofyear_sin_5,datetime_Dayofyear_cos_1,datetime_Dayofyear_cos_2,datetime_Dayofyear_cos_3,datetime_Dayofyear_cos_4,datetime_Dayofyear_cos_5
0,2010-01-01 00:00:00+07:00,55.0,29.346000,24.346000,26.493912,33.960170,28.426277,30.544830,44896.137,25858.787,4.000000,14.0,15.978486,38.160000,245.528370,16.24,3.553079,Drizzle: Dense,2010-01-01,2010,January,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,1,0,1,1,4,4,1,1262278800,5.000000e-01,8.660254e-01,1.000000e+00,8.660254e-01,5.000000e-01,0.866025,0.5,6.123234e-17,-0.5,-0.866025,1.000000e+00,1.224647e-16,-1.000000e+00,-2.449294e-16,1.000000e+00,6.123234e-17,-1.0,-1.836970e-16,1.0,3.061617e-16,-8.660254e-01,8.660254e-01,-4.898587e-16,-8.660254e-01,8.660254e-01,-0.5,-0.5,1.0,-0.5,-0.5,0.017166,0.034328,0.051479,0.068615,0.085731,0.999853,0.999411,0.998674,0.997643,0.996318
1,2010-01-02 00:00:00+07:00,63.0,30.696000,23.896000,26.858500,37.097730,28.470013,31.728783,44890.758,32148.900,6.500000,4.0,14.707222,31.319998,237.893750,19.80,4.135429,Rain: Moderate,2010-01-02,2010,January,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,4.000000,4.000000,1,1,0,0,0,0,0,5,5,2,1262365200,5.000000e-01,8.660254e-01,1.000000e+00,8.660254e-01,5.000000e-01,0.866025,0.5,6.123234e-17,-0.5,-0.866025,1.000000e+00,1.224647e-16,-1.000000e+00,-2.449294e-16,1.000000e+00,6.123234e-17,-1.0,-1.836970e-16,1.0,3.061617e-16,-8.660254e-01,-8.660254e-01,2.388680e-15,8.660254e-01,8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5,0.034328,0.068615,0.102821,0.136906,0.170830,0.999411,0.997643,0.994700,0.990584,0.985301
2,2010-01-03 00:00:00+07:00,63.0,30.346000,24.246000,26.652250,36.627003,28.546375,31.245909,44884.926,38630.723,7.600000,11.0,12.646200,26.640000,243.816120,17.32,3.662728,Rain: Moderate,2010-01-03,2010,January,6.500000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.750000,5.250000,6.000000,1,1,0,0,0,0,0,6,6,3,1262451600,5.000000e-01,8.660254e-01,1.000000e+00,8.660254e-01,5.000000e-01,0.8660

In [23]:
os.makedirs("data/02_processed/", exist_ok=True)

daily_processed_df.to_csv("data/02_processed/jakarta_daily_weather_processed.csv", index=False)